In [4]:
import sys
import os

# Récupérer le chemin absolu du dossier parent
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Ajouter ce chemin aux modules Python
sys.path.append(parent_dir)
print(parent_dir)

c:\Users\manon\OneDrive\Bureau\X\bioinf_MEA_stacking\MEA_stacking
